In [25]:
import ipaddress
import math

def calculate_subnets(network, num_departments):
    base_network = ipaddress.ip_network(network, strict=False)
    prefix_length = base_network.prefixlen
    max_subnets = 2 ** (base_network.max_prefixlen - prefix_length)

    if num_departments > max_subnets:
        raise ValueError("Nombre de départements trop élevé pour cette plage d'adresses réseau !")

    new_prefix = prefix_length + math.ceil(math.log2(num_departments))
    subnets = list(base_network.subnets(new_prefix=new_prefix))[:num_departments]

    result = []
    for i, subnet in enumerate(subnets):
        first_ip = subnet.network_address + 1
        last_ip = subnet.broadcast_address - 1
        result.append({
            "Dpt": f"Dpt {i + 1}",
            "Sous-reseau": str(subnet.network_address),
            "Intervalle": f"{first_ip} - {last_ip}",
            "Diffusion": str(subnet.broadcast_address),
            "Masque": str(subnet.netmask),
        })
    return result

def allocate_machines(network, department_data):
    base_network = ipaddress.ip_network(network, strict=False)
    prefix_length = base_network.prefixlen

    department_data = sorted(department_data, key=lambda x: x["machines"], reverse=True)
    subnets = []

    for dep in department_data:
        required_ips = dep["machines"] + 2  # Machines + réseau + diffusion
        new_prefix = base_network.max_prefixlen - math.ceil(math.log2(required_ips))
        subnet = next(base_network.subnets(new_prefix=new_prefix))
        base_network = ipaddress.ip_network((subnet.network_address + subnet.num_addresses, subnet.prefixlen), strict=False)

        first_ip = subnet.network_address + 1
        last_ip = subnet.broadcast_address - 1

        subnets.append({
            "Dpt": dep["name"],
            "Sous-reseau": str(subnet.network_address),
            "Intervalle": f"{first_ip} - {last_ip}",
            "Diffusion": str(subnet.broadcast_address),
            "Masque": str(subnet.netmask),
        })
    return subnets

def display_table(data):
    print("\nTableau des sous-réseaux :")
    print("Dpt".ljust(15), "Sous-reseau".ljust(20), "Intervalle".ljust(30), "Diffusion".ljust(20), "Masque")
    print("-" * 95)
    for entry in data:
        print(entry["Dpt"].ljust(15), entry["Sous-reseau"].ljust(20), entry["Intervalle"].ljust(30), entry["Diffusion"].ljust(20), entry["Masque"])

def main():
    print("Plan d'adressage IP")
    print("1. Sous-adressage basé sur le nombre de départements")
    print("2. Plan d'adressage en fonction des besoins des départements")
    choice = input("Choisissez une option (1/2) : ").strip()

    if choice == "1":
        num_departments = int(input("Entrez le nombre de départements : "))
        network = input("Entrez l'adresse réseau (ex: 192.168.1.0/24) : ")

        try:
            data = calculate_subnets(network, num_departments)
            display_table(data)
        except ValueError as e:
            print(f"Erreur : {e}")

    elif choice == "2":
        num_departments = int(input("Entrez le nombre de départements : "))
        departments = []

        for i in range(num_departments):
            machines = int(input(f"Entrez le nombre de machines pour le département {i + 1} : "))
            departments.append({"name": f"Dpt {i + 1}", "machines": machines})

        network = input("Entrez l'adresse réseau (ex: 10.0.0.0/8) : ")

        try:
            data = allocate_machines(network, departments)
            display_table(data)
        except ValueError as e:
            print(f"Erreur : {e}")

    else:
        print("Choix invalide, veuillez réessayer.")

if __name__ == "__main__":
    main()


Plan d'adressage IP
1. Sous-adressage basé sur le nombre de départements
2. Plan d'adressage en fonction des besoins des départements


Choisissez une option (1/2) :  2
Entrez le nombre de départements :  1
Entrez le nombre de machines pour le département 1 :  1500
Entrez l'adresse réseau (ex: 10.0.0.0/8) :  10.0.0.0/8



Tableau des sous-réseaux :
Dpt             Sous-reseau          Intervalle                     Diffusion            Masque
-----------------------------------------------------------------------------------------------
Dpt 1           10.0.0.0             10.0.0.1 - 10.0.7.254          10.0.7.255           255.255.248.0
